# Unstructured Text Analysis
## Sarah Samuel, cs344

In [8]:
import pandas as pd
import numpy as np
import string
import codecs
import nltk
import os
import re
from nltk.corpus import stopwords

In [10]:
# read in csv

uta_df = pd.read_csv("historicalData5years.csv")
uta_df.head()

,TcaPkey,TcaCompletedDate,TcaProblem,Part
0,708225,00:00.0,Keeps rebooting like before. See ticket #698931.,POWER SUPPLY
1,710945,20:00.0,Library controller will not boot with RAID car...,CONTROLLER
2,710945,20:00.0,Library controller will not boot with RAID car...,MONITOR
3,711988,30:00.0,DS4300 TOP - Failed controler and failed contr...,BATTERY
4,711988,30:00.0,DS4300 TOP - Failed controler and failed contr...,CONTROLLER


In [22]:
#create a list of stopwords
Project
nltk.download('stopwords')
stop_words = stopwords.words('english') 
more_stop = ['gb','needs','consume','appears','service','ticket',
             'code','x','parts','model','pn','part','number',
             'need','onsite','spare','last','part','number','week',
             'would','like','we','judy','still','stays','stay',
             'doesnt','getting','keeps','back','replaced','two',
             'one','three','g','call','home','yesterday','today',
             'itll','think','customer','ts','noc','thinks','minutes',
             'thought','e','hi','k','gb','call','need','needs','please']
stop_words.extend(more_stop)
print(len(stop_words))

235


[nltk_data] Downloading package stopwords to /home/ss63/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
#clean up TcaProblem field and only keep words that are not in stop_words

uta_df['TcaProblem'] = uta_df['TcaProblem'].astype(str)
uta_df.TcaProblem = uta_df['TcaProblem'].str.lower()
uta_df['TcaProblem'] = uta_df['TcaProblem'].str.replace('[{}]'.format(string.punctuation), '')
uta_df['TcaProblem'] = uta_df['TcaProblem'].str.replace('\d+', '')
uta_df['TcaProblem'] = uta_df['TcaProblem'].apply(lambda x: [item for item in x.split() if item not in stop_words])

In [15]:
#reformat cleaned up TcaProblem field
uta_df['TcaProblem'] = uta_df['TcaProblem'].astype(str)
uta_df['TcaProblem'] = uta_df['TcaProblem'].str.replace('[{}]'.format(string.punctuation), '')

In [16]:
#create an n-gram list and add it to the dataframe

ngram_list = list()
for x in uta_df.TcaProblem:
    ngram_words = list(nltk.ngrams(x.split(), 2)) #change 3 to any sensible positive int.
    ngram_list.append(ngram_words)
uta_df['Ngrams'] = ngram_list

In [19]:
#places n-gram tuples into list, when part is index

uta_df2 = pd.DataFrame(uta_df['Ngrams'].values.tolist(), index= uta_df.Part)
stack = uta_df2.stack()
uta_stacked_df = pd.DataFrame(stack)

uta_df3 = uta_stacked_df.reset_index()
uta_df3.columns = ['Part','Count', 'Ngrams']
print(uta_df3.head())

           Part  Count                 Ngrams
0  POWER SUPPLY      0       (rebooting, see)
1    CONTROLLER      0  (library, controller)
2    CONTROLLER      1     (controller, boot)
3    CONTROLLER      2           (boot, raid)
4    CONTROLLER      3           (raid, card)


In [20]:
#group by Part, then Ngrams and rank by most common

uta_df3_grouped = uta_df3.groupby(['Part', 'Ngrams'], as_index=False).count()
uta_df3 = uta_df3_grouped.sort_values(by = ['Part','Count'], ascending = False)

uta_df3 = pd.DataFrame(uta_df3)Project
uta_df3 = uta_df3.groupby('Part').head(15)

In [25]:
#place single ticket into a list and parse the problem into bi-grams

tca_problem = ["Flashing red light failed drive"]

for x in tca_problem:
    ngram_words = list(nltk.ngrams(x.split(), 2)) 
ngram_words

#place the parsed ticket into a dataframe
single_ticketdf = pd.DataFrame()
single_ticketdf['Ngrams'] = ngram_words
print(single_ticketdf)

            Ngrams
0  (Flashing, red)
1     (red, light)
2  (light, failed)
3  (failed, drive)


In [27]:
'''
Join the parsed single-ticket df onto the bi-gram df. This associates the bi-grams of the single-ticket
to hardware parts in the unstrucdf_3 df and then predicts which hardware part should be used to fix the issue
'''
merge_df = pd.merge(uta_df3,single_ticketdf, on='Ngrams', how='inner')
Project
prop = merge_df['Count']/sum(merge_df['Count'])
merge_df['Probability'] = prop
print(merge_df)


         Part           Ngrams  Count  Probability
0     MONITOR     (red, light)      1     0.012048
1  DISK DRIVE  (failed, drive)     82     0.987952
